# Import Packages

In [ ]:
import math
import pandas as pd
import torch

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import schedulefree

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np

In [ ]:
# Not needed for now since these models are so small they run well on CPU
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available()  else 'cpu')
#device

# Import and Preprocess the data

In [ ]:
def import_data(path):
    df = pd.read_csv(path)
    df = df.rename(columns={'Date': 'date','Open':'open','High':'high','Low':'low',
                            'Close':'close','Adj Close':'adj_close','Volume':'volume'})
    return df
    

In [ ]:
def preprocess_data(df: pd.DataFrame):
    if df.isnull().values.sum() > 0:
        print("Uh oh, null values")
        print(df.isnull().values.sum())
    if df.isna().values.any():
        print("NA values")

    # convert date field from string to Date format
    df['date'] = pd.to_datetime(df.date)
    return df

In [ ]:
def plot_open_close_prices(df:pd.DataFrame):
    fig = px.line(maindf,
              x=maindf['date'], 
              y=[maindf['open'],
                maindf['close']],        
              labels={'value':'Stock price','date': 'Date'})
    fig.show()

In [ ]:
#maindf = import_data("data/NVDA.csv")
#maindf = import_data("data/TSLA.csv")
maindf = import_data("data/AAPL.csv")
maindf = preprocess_data(maindf)
plot_open_close_prices(maindf)

In [ ]:
def prepare_data(df: pd.DataFrame):
    closedf = np.array(maindf['close']).reshape(-1,1)
    print(f"last: {closedf[-1]}")
    scaler=MinMaxScaler(feature_range=(0,1))
    closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
    print(f"last scaled: {closedf[-1]}")
    return closedf, scaler
    

In [ ]:
def split_data(data: np.array, train_pct = 0.8):
    data_len = data.shape[0]
    train_size = int(data_len * train_pct)
    
    train_data = data[0:train_size,:]
    val_data = data[train_size:,:]
    print(f"train shape: {train_data.shape},  val shape: {val_data.shape}")
    return train_data, val_data
    

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step - 1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    # convert to numpy arrays
    dataX = np.array(dataX)
    dataY = np.array(dataY)
    # reshape input to be [samples, time steps, features] which is required for LSTM
    dataX = np.expand_dims(dataX, axis=2)
    dataY = np.expand_dims(dataY, axis=1)
    # convert to pytorch tensors
    dataX = torch.from_numpy(dataX).type(torch.Tensor)
    dataY = torch.from_numpy(dataY).type(torch.Tensor)
    return dataX, dataY

In [ ]:
closedf, scaler = prepare_data(maindf)
train_data, val_data = split_data(closedf)
print(f"last val_data: {val_data[-1]}")
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_val, y_val = create_dataset(val_data, time_step)
print(y_val[-1])
print(f"X_train shape {X_train.shape},  y_train shape {y_train.shape}")
print(f"X_val shape {X_val.shape},  y_val shape {y_val.shape}")

# Create the Model

In [ ]:
def show_model_inf(model):
    print("----------- model information -----------")
    print(model)
    print(f"number of layers: {len(list(model.parameters()))}")
    for i in range(len(list(model.parameters()))):
        print 
        print(list(model.parameters())[i].size())
        
    # total parameters and trainable parameters
    total_params = sum(p.numel() for p in model.parameters())
    print(f"{total_params:,} total parameters.")
    total_trainable_params = sum(
        p.numel() for p in model.parameters() if p.requires_grad)
    print(f"{total_trainable_params:,} training parameters.\n")

In [ ]:


# Here we define our model as a class
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.num_layers = num_layers

        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 32, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out


# Train the Model

In [ ]:
def train_model(model, optimiser, num_epochs = 200):
    loss_fn = torch.nn.MSELoss()
    # Train model
    hist = { 
        "loss" : np.zeros(num_epochs),
        "val_loss" : np.zeros(num_epochs)
    }
    best_val_loss = float('inf')
    BEST_MODEL_PATH = "best_model.mod"

    for t in range(num_epochs):
        # Forward pass
        model.train()
        optimiser.train()
        y_train_pred = model(X_train)

        loss = loss_fn(y_train_pred, y_train)
        hist["loss"][t] = loss.item()

        # Zero out gradient, else they will accumulate between epochs
        optimiser.zero_grad()
        # Backward pass
        loss.backward()
        # Update parameters
        optimiser.step()

        # validation
        model.eval()
        optimiser.eval()
        y_val_pred = model(X_val)
        val_loss = loss_fn(y_val_pred, y_val)
        hist["val_loss"][t] = val_loss.item()
        if val_loss < best_val_loss:
            best_val_loss = val_loss
        #print(f"saving model at {t}, loss {val_loss}")
            torch.save({
                'epoch': t,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimiser.state_dict(),
                'loss': val_loss
                }, BEST_MODEL_PATH)


        if t % 20 == 0 and t !=0:
            #print("Epoch ", t, "train MSE: ", loss.item())
            print(f"Epoch {t}  train MSE {loss.item():2.5f}  val MSE {val_loss.item():2.5f}")

    print(f"Epoch {t}  train MSE {loss.item():2.5f}  val MSE {val_loss.item():2.5f}")
    return model, optimiser, hist

In [ ]:
def load_best_model(model, optimiser, path):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimiser.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    model.eval()
    print(f"Best model loaded from epoch {epoch}, loss {loss}")
    return model, optimiser

In [ ]:
def show_training_loss(hist):
    df = pd.DataFrame(hist)
    fig = px.line(df, 
              y=[df['loss'],
                 df['val_loss']],                                       
              labels={'value':'epoch','value': 'loss'})
    fig.show()


In [ ]:
# create the model
input_dim = 1
hidden_dim = 32
num_layers = 2 
output_dim = 1
torch.manual_seed(42)
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
show_model_inf(model)

# train the model
#optimiser = torch.optim.Adam(model.parameters(), lr=0.02)
optimizer = schedulefree.AdamWScheduleFree(model.parameters(), lr=0.02)
epochs = 600
model, optimiser, hist = train_model(model, optimizer, num_epochs = epochs)
show_training_loss(hist)

# Model Evaluation

In [ ]:
# model eval
model, optimiser = load_best_model(model, optimiser, "best_model.mod")

### Lets Do the prediction and check performance metrics
train_predict=model(X_train)
val_predict=model(X_val)

In [ ]:
# convert back to original data form
train_predict = scaler.inverse_transform(train_predict.detach().numpy())
val_predict = scaler.inverse_transform(val_predict.detach().numpy())
original_ytrain = scaler.inverse_transform(y_train.detach().numpy())
original_yval = scaler.inverse_transform(y_val.detach().numpy()) 

## Compare original closing prices with predicted

In [ ]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:,:] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = val_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': maindf['date'],
                       'original_close': maindf['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True)
fig.show()

## For fun lets predict the next 20 days

In [ ]:
pred_days = 20
# store last 15 test samples in a list
x_input_list = list(closedf[-time_step:].squeeze())
preds = np.zeros(pred_days)

for i in range(pred_days):
    # convert the input list to a tensor of the correct dimensions
    x_input = torch.FloatTensor(x_input_list).unsqueeze(dim=0).unsqueeze(dim=2)
    # make a prediction
    yhat = model(x_input)
    # convert the tensor back to float
    yhat_float = float(yhat.detach().numpy()[0][0])
    preds[i] = yhat_float
    # remove the first element of the list and add the prediction
    x_input_list = x_input_list[1:]
    x_input_list.append(yhat_float)


preds_next_20 = scaler.inverse_transform(preds.reshape(1,-1))

In [ ]:
prev_20 = np.zeros(pred_days * 2)
prev_20[:] = np.nan
prev_20[:pred_days] = maindf['close'][-pred_days:].squeeze()
next_20 = np.zeros(pred_days * 2)
next_20[:] = np.nan
next_20[pred_days:] = preds_next_20

df = pd.DataFrame({
    "last 20" : prev_20,
    "next 20" : next_20
})

fig = px.line(df, y=[df['last 20'],
                     df['next 20']],
              labels={'value':'Stock price','date': 'Date'})

fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True)
fig.show()

## Take a look at several standard metrics

### RMSE, MSE and MAE

In [ ]:
# Evaluation metrices RMSE, MSE and MAE
print("-------------------------------------------------------------------------------------")
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Val data RMSE: ", math.sqrt(mean_squared_error(original_yval,val_predict)))
print("Val data MSE: ", mean_squared_error(original_yval,val_predict))
print("Val data MAE: ", mean_absolute_error(original_yval,val_predict))

### R-squared (R2) 
is a statistical measure that represents the proportion of the variance for a dependent variable thats 
explained by an independent variable or variables in a regression model.

1 = Best

0 or < 0 = worse

In [ ]:
#R-squared (R2) is a statistical measure that represents the proportion of the variance for a dependent variable thats 
#explained by an independent variable or variables in a regression model.
#
# 1 = Best
# 0 or < 0 = worse
print("--------- R2 score ------------------------------------------------------------------")
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Val data R2 score:", r2_score(original_yval, val_predict))

### Explained variance regression score
The explained variance score explains the dispersion of errors of a given dataset, and the formula is written as follows: Here, and Var(y) is the variance of prediction errors and actual values respectively. Scores close to 1.0 are highly desired, indicating better squares of standard deviations of errors.

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_yval, val_predict))

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_yval, val_predict))

## Additional metrics

In [ ]:
orig_test = original_yval.squeeze()
pred_test = val_predict.squeeze()

### holding through the test period

In [ ]:
hold_value = orig_test[-1] - orig_test[0]
hold_change = hold_value * 100 / orig_test[0]
print(f"value change {hold_value:.2f}, percent change {hold_change:.2f}, num days {len(orig_test)}")

### going long/short on the next day from the prediction

In [ ]:
value = 0
for i in range(len(pred_test)):
    if i == 0: continue
    if pred_test[i] > orig_test[i-1]:
        value += orig_test[i] - orig_test[i-1]
    else:
        value -= (orig_test[i] - orig_test[i-1])

percent = value * 100 / orig_test[0]
print(f"value change {value:.2f}, percent change {percent:.2f}, num days {len(orig_test)}")

# Conclusion

The results look fairly interesting. In this instance the model seems to do fairly well and outperforms the buy and hold method through the test period. I wouldn't put any real money in this yet.
 1. The comparison assumes you could buy or short the stock on the previous days close and sell it at the next days close.
 2. I'm not really sure the best or most realistic way to evaluate a model's performance for financial prediction. Essentially the investor wants to achieve alpha or outperformance.
 3. The model performs pretty well in this instance, but I've seen instances where it does poorly.

There are many things different things to try.
 1. A larger or deeper LSTM model.
 2. Train on more data.
 3. Use longer or shorter periods than the 15 day sequence for training.
 4. Use more variables than just closing price.

It also might be interesting to
 1. See how well this model does on other stocks.
 2. Train a base model on several different stocks and see if finetuning for individual stocks goes much faster.
 3. Try to quantify how well the metrics like the R2 score translates to performance